In [ ]:
import contextlib
import random

from keyname import keyname as kn
from matplotlib import pyplot as plt
from nbmetalog import nbmetalog as nbm
import numpy as np
import pandas as pd
import patchworklib as pw
from scipy import stats
import seaborn as sns
from slugify import slugify
from teeplot import teeplot as tp
import itertools as it
import matplotlib.patheffects as pe


In [ ]:
np.random.seed(1)
random.seed(1)


In [ ]:
from conduitpylib.viz import (
    runtime_indicators_concat_regplot_pairplot,
    runtime_indicators_pairplot,
    runtime_indicators_regplot,
)
from conduitpylib.wrangle import (
    retrieve_and_prepare_delta_dataframes,
    wrangle_instrumentation_longform,
)
from _rename_allocation import rename_allocation


In [ ]:
nbm.print_metadata()


# Get Data


In [ ]:
longitudinal_df, snapshot_df = retrieve_and_prepare_delta_dataframes(
    df_inlet_url="https://osf.io/jgpnv/download",
    df_outlet_url="https://osf.io/ncdfq/download",
    treatment_column="Allocation",
)


# Compare Finalized and Rolling Measures


In [ ]:
data = wrangle_instrumentation_longform(
    longitudinal_df=longitudinal_df,
    snapshot_df=snapshot_df,
)

patchwork_context = pw.patched_axisgrid
for context in contextlib.nullcontext, patchwork_context:
    for allocation, group in data.groupby("Allocation"):
        plt.clf()
        with context():
            group_df = group.reset_index()
            regplot_facetgrid = tp.tee(
                runtime_indicators_regplot,
                data=rename_allocation(group_df),
                compare="Instrumentation",
                comparison_x="Longitudinal",
                comparison_y="Snapshot",
                teeplot_outattrs={
                    "allocation": slugify(allocation),
                },
                teeplot_save=(context is not patchwork_context),
            )
            pairplot_grid = tp.tee(
                runtime_indicators_pairplot,
                data=rename_allocation(group_df),
                hue="Instrumentation",
                teeplot_outattrs={
                    "allocation": slugify(allocation),
                },
                teeplot_save=(context is not patchwork_context),
            )

            if context is patchwork_context:
                tp.tee(
                    runtime_indicators_concat_regplot_pairplot,
                    regplot_facetgrid,
                    pairplot_grid,
                    teeplot_outattrs={
                        "allocation": slugify(allocation),
                    },
                )

            plt.show()
            plt.clf()
